In [22]:
import os
import time
import platform
import requests
import browser_cookie3
from bs4 import BeautifulSoup

def load_cookie_from_chrome():
    global jar

    cookie_path = None
    os_type = platform.system()
    if os_type == "Linux":
        # browser_cookie3 also checks beta version of google chrome's cookie file.
        cookie_path = os.path.expanduser(
            "~/.config/google-chrome/Default/Cookies")
        if os.path.exists(cookie_path) is False:
            cookie_path = os.path.expanduser(
                "~/.config/google-chrome-beta/Default/Cookies")
    elif os_type == "Darwin":
        cookie_path = os.path.expanduser(
            "~/Library/Application Support/Google/Chrome/Default/Cookies")
    elif os_type == "Windows":
        cookie_path = os.path.expandvars(
            "%LOCALAPPDATA%/Google/Chrome/User Data/Default/Cookies")
    else:  # Jython?
        print("지원하지 않는 환경입니다.")
        close()

    if os.path.exists(cookie_path) is False:
        print("기본 쿠키 파일 경로에 파일이 존재하지 않습니다. 아래 링크를 참조하여 쿠키 파일 경로를 지정해주세요.")
        close()

    jar = browser_cookie3.chrome(domain_name="foresttrip.go.kr")

check_date_list = [
    ["20210821", "20210822", "토", "일"],
    ["20210828", "20210829", "토", "일"],
    ["20210904", "20210905", "토", "일"],
    ["20210911", "20210912", "토", "일"],
    ["20210918", "20210919", "토", "일"]
]

hmpgId = "ID02030052"
keyword = "비학산자연휴양림"
indvz_url = "https://www.foresttrip.go.kr/indvz/main.do?hmpgId="+hmpgId
check_url = "https://www.foresttrip.go.kr/rep/or/sssn/fcfsRsrvtPssblGoodsDetls.do"
net_funnel_url = "https://nf.foresttrip.go.kr/ts.wseq?opcode=5101&nfid=0&prefix=NetFunnel.gRtype=5101;&sid=service_1&aid=action2&js=yes"
header = {
    "Accept": "*/*",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "ko,ko-KR;q=0.9,en;q=0.8",
    "Host": "www.foresttrip.go.kr",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"
}
load_cookie_from_chrome()
print("예약가능여부 체크중...")
main_page_result = requests.get(indvz_url, cookies=jar)
soup = BeautifulSoup(main_page_result.text, "html5lib")
_csrf = soup.select("input[name='_csrf']")[0].attrs["value"]
try_count = 0
done = False

while done == False:
    time.sleep(5)
    try_count += 1
    print("{}번쨰 체크중...".format(try_count), end="\r")
    for check_date in check_date_list:
        this_net_funnel_result = requests.get(net_funnel_url, cookies=jar)
        this_net_funnel_cookie = this_net_funnel_result.text.split(";")[1].split("='")[1][:-1]
        this_cookie = browser_cookie3.create_cookie("www.foresttrip.go.kr", "/", 0, 0, "NetFunnel_ID", this_net_funnel_cookie)
        jar.set_cookie(this_cookie)
        ymd_str1 = check_date[0][2:4]+"/"+check_date[0][4:6]+"/"+check_date[0][6:8]+"("+check_date[2]+")"
        ymd_str2 = check_date[1][2:4]+"/"+check_date[1][4:6]+"/"+check_date[1][6:8]+"("+check_date[3]+")"
        this_check_param = {
            "_csrf" : _csrf,
            "hmpgId" : hmpgId,
            "srchInsttId" : hmpgId,
            "srchRsrvtBgDt" : check_date[0],
            "srchRsrvtEdDt" : check_date[1],
            "srchUseDt" : ymd_str1+"-"+ymd_str2,
            "srchSthngCnt" : "1",
            "srchStngNofpr" : "2",                
            "rsrvtWtngSctin" : "01",
            "cmdogYn" : "N",
            "bbqYn" : "N",
            "dsprsYn" : "N",
            "otsdWeterYn" : "N",
            "wifiYn" : "N",
            "snowPlaceYn" : "N",
            "gNowPage" : "1",
            "goodsClsscHouseCdArr" : "01001",
            "keyword" : keyword,
            "calPicker" : ymd_str1+"-"+ymd_str2
        }
        this_check_result = requests.get(check_url, data=this_check_param, headers=header, cookies=jar)
        print(this_check_result.text)
        break
    break

예약가능여부 체크중...
Not Acceptable
